# Moments, Barriers, and Classical Control in Cirq

This notebook teaches:
- What a Cirq Moment is and how automatic packing works.
- Why classical controls (measurement-key–conditioned ops) may not reshape moments by themselves.
- Techniques to enforce or visualize phase boundaries: explicit moments, identity layers, custom barrier gates.

## 1. Imports

In [ ]:
import cirq
from typing import Sequence

## 2. Core Concepts

A `cirq.Moment` is a parallel layer of operations with no shared qubits. Cirq packs a flat list of operations greedily into moments based only on qubit conflicts (and basic measurement ordering), not on classical-control dependencies.

Classically controlled operations (e.g. `op.with_classical_controls('key')`) use measurement outcomes at execution time, but they do not automatically create a separate moment in the printed diagram. Therefore, if you need a guaranteed visual or structural boundary between measurements and feed-forward corrections, you must create one explicitly.

First, we will define some custom gates to perform the teleportation algorithm. Some of these were seen in the custom gates notebook.

In [ ]:
import cirq
from typing import Sequence

# --- Helper: Bell pair preparation ---
class BellPairGate(cirq.Gate):
    def _num_qubits_(self): return 2
    def _decompose_(self, qs):
        c, t = qs
        yield cirq.H(c)
        yield cirq.CNOT(c, t)
    def _circuit_diagram_info_(self, args): return 'BP','BP'
BP = BellPairGate()

# --- Helper: Bell-basis prep between message and Alice ---
class MessagePrepGate(cirq.Gate):
    def _num_qubits_(self): return 2
    def _decompose_(self, qs):
        m, a = qs
        yield cirq.CNOT(m, a)
        yield cirq.H(m)
    def _circuit_diagram_info_(self, args): return 'ME_M','ME_A'
MPrep = MessagePrepGate()

# --- Feed-forward corrections (decomposes to classically controlled X/Z) ---
class MessageCorrection(cirq.Gate):
    def __init__(self, b1: str, b2: str):
        self.b1 = b1; self.b2 = b2
    def _num_qubits_(self): return 1
    def _decompose_(self, qs):
        (bob,) = qs
        yield cirq.Z(bob).with_classical_controls(self.b1)
        yield cirq.X(bob).with_classical_controls(self.b2)
    def _circuit_diagram_info_(self, args): return f'Z^{self.b1}X^{self.b2}'

Next, we will define the three different ways of putting the algorithm together, one with automatic packing, one with using the barrier gate, and one with explicit moments.

In [ ]:
# --- Optional full-qubit barrier (visual layer separator) ---
class BarrierGate(cirq.Gate):
    def __init__(self, num_qubits: int): self._n = num_qubits
    def _num_qubits_(self): return self._n
    def _decompose_(self, qs): return []  # no-op
    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(wire_symbols=('│',) * self._n)

# Teleportation ops as a flat list (auto packing may merge corrections upward if not separated)
def teleport_ops_with_optional_barrier(use_barrier: bool) -> Sequence[cirq.Operation]:
    alice = cirq.NamedQubit('alice')
    bob = cirq.NamedQubit('bob')
    msg = cirq.NamedQubit('msg')
    ops: Sequence[cirq.Operation] = []
    ops.append(BP.on(alice, bob))
    ops.append(cirq.rx(0.7)(msg))
    ops.append(MPrep.on(msg, alice))
    ops.append(cirq.measure(msg, key='b1'))
    ops.append(cirq.measure(alice, key='b2'))
    if use_barrier:
        ops.append(BarrierGate(3).on(msg, alice, bob))
    ops.append(MessageCorrection('b1','b2').on(bob))
    ops.append(cirq.measure(bob, key='result'))
    return ops

# Explicit-moment construction for guaranteed boundaries
def teleport_explicit_moments() -> cirq.Circuit:
    alice = cirq.NamedQubit('alice')
    bob = cirq.NamedQubit('bob')
    msg = cirq.NamedQubit('msg')
    return cirq.Circuit([
        cirq.Moment([cirq.H(alice)]),
        cirq.Moment([cirq.CNOT(alice, bob), cirq.rx(0.7)(msg)]),
        cirq.Moment([cirq.CNOT(msg, alice)]),
        cirq.Moment([cirq.H(msg)]),
        cirq.Moment([cirq.measure(msg, key='b1'), cirq.measure(alice, key='b2')]),
        cirq.Moment([cirq.Z(bob).with_classical_controls('b1')]),
        cirq.Moment([cirq.X(bob).with_classical_controls('b2')]),
        cirq.Moment([cirq.measure(bob, key='result')]),
    ])

## 3. Auto-Packed vs. Barrier vs. Explicit Moments
We construct three circuits:
1. Auto-packed (no barrier): may merge feed-forward corrections earlier.
2. Auto-packed with barrier: forces a visible layer boundary.
3. Explicit moments: manually defined phases, independent of greedy packing.

In [ ]:
auto_no_barrier = cirq.Circuit(teleport_ops_with_optional_barrier(False))
auto_with_barrier = cirq.Circuit(teleport_ops_with_optional_barrier(True))
explicit = teleport_explicit_moments()

print("Auto-packed (no barrier):")
print(auto_no_barrier)
print("\nAuto-packed (with barrier):")
print(auto_with_barrier)
print("\nExplicit moments:")
print(explicit)

## 5. Simulation
We simulate the explicit-moment circuit to sample Bob's teleported state |ψ> = Rx(0.7)|0>. Expected probabilities: P(0) ≈ cos^2(0.35) ≈ 0.88, P(1) ≈ sin^2(0.35) ≈ 0.12.

In [ ]:
import math

sim = cirq.Simulator()
result = sim.run(auto_with_barrier, repetitions=1000)
print("Bob's teleported state measurement histogram (expected ~88% zeros, ~12% ones):")
print(result.histogram(key='result'))
# Expected probabilities
p0 = math.cos(0.35)**2
p1 = math.sin(0.35)**2
print(f"Expected P(0)≈{p0:.3f}, P(1)≈{p1:.3f}")